In [6]:
#@title Install required packages
!uv pip install -U pse # proxy structuring engine
!uv pip install sentencepiece
!uv pip install accelerate
!uv pip install transformers
!uv pip install torch
!uv pip install numpy
!uv pip install bitsandbytes
!uv pip install sentencepiece
!uv pip install protobuf
!uv pip install -U tqdm
!uv pip install ipywidgets

Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
⠙ Resolving dependencies...                                                     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 26 packages in 95ms                                         
Audited 26 packages in 0.09ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 2ms


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Resolved 1 package in 47ms                                           
Audited 1 package in 0.08ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
# @title Setup Llama 3.2 1B and create the StructuringEngine
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

from pse.engine.structuring_engine import StructuringEngine
from pse.util.torch_mixin import PSETorchMixin


class PSE_Torch(PSETorchMixin, LlamaForCausalLM):
    pass


model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = PSE_Torch.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.config.pad_token_id = model.config.eos_token_id[0]
if model.generation_config:
    model.generation_config.top_p = None
    model.generation_config.top_k = 8
    model.generation_config.do_sample = True
    model.generation_config.temperature = 0.9
    model.generation_config.pad_token_id = model.config.eos_token_id[0]
    model.generation_config.max_new_tokens = 1000

model.engine = StructuringEngine(tokenizer)

In [8]:

#@title test simple json object generation
import json

SIMPLE_JSON_SCHEMA = {
    "type": "object",
    "properties": {"value": {"type": "number"}},
    "required": ["value"],
}
model.engine.configure(SIMPLE_JSON_SCHEMA)
prompt = (
    "Please generate a json object with the value 9.11, with the following schema:\n"
)
prompt += json.dumps(SIMPLE_JSON_SCHEMA, indent=2)

messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
output = model.generate(
    input_ids,
    do_sample=True,
)
# you can print the prompt + output:
#   print(tokenizer.decode(output[0]))
# you can also access just the structured output:
#   engine.parse_structured_output()
structured_output = model.engine.parse_structured_output(output_type=dict)
print(100 * "-")
print(json.dumps(structured_output, indent=2))

----------------------------------------------------------------------------------------------------
{
  "value": 9.11
}


In [9]:
# @title Test advanced-json generation
ADVANCED_JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {"const": "metacognition"},
        "arguments": {
            "type": "object",
            "properties": {
                "chain_of_thought": {
                    "type": "array",
                    "description": "A sequence of high-level thoughts, reasoning and internal dialogue.\nThe chain of thought should be a sequence of thoughts that are related to the question.\n",
                    "items": {
                        "type": "string",
                        "minLength": 20,
                        "maxLength": 200,
                    },
                    "minItems": 1,  # floor the number of thoughts
                    "maxItems": 3,  # limit the number of thoughts
                },
            },
            "required": ["chain_of_thought"],
        },
    },
    "required": ["name", "arguments"],
}
model.engine.configure(ADVANCED_JSON_SCHEMA)
raw_prompt = (
    f"This is a test of your abilities."
    f"Please format your response to follow the following schema:\n{json.dumps(ADVANCED_JSON_SCHEMA, indent=2)}\n"
)
messages = [{"role": "user", "content": raw_prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
greedy_output = model.generate(
    input_ids,
    do_sample=True,
)
structured_output = model.engine.parse_structured_output(output_type=dict)
print(100 * "-")
print(json.dumps(structured_output, indent=2))


----------------------------------------------------------------------------------------------------
{
  "arguments": {
    "chain_of_thought": [
      "The process of thinking about one's own thought processes. It involves analyzing and evaluating our internal dialogue to improve our metacognition.\n",
      "A tool for reflecting on our thoughts, feelings, and behaviors to gain a deeper understanding of ourselves.\n",
      "A mindset that enables us to think critically and intentionally, recognizing the role of our thoughts and emotions in shaping our experiences.\n"
    ]
  },
  "name": "metacognition"
}


In [10]:
# @title Test pydantic generation
from pydantic import BaseModel


class CursorPositionModel(BaseModel):
    """
    An object representing the position and click state of a cursor.

    Attributes:
        x_pos: The horizontal position of the cursor in pixels
        y_pos: The vertical position of the cursor in pixels
        left_click: Whether the left mouse button is currently pressed. Default is False.
    """

    x_pos: int
    y_pos: int
    left_click: bool = False


json_schema: dict = model.engine.configure(
    CursorPositionModel, json_delimiters=("<cursor>", "</cursor>")
)
prompt = (
    "Please use the following schema to generate a cursor position:\n"
    f"{json.dumps(json_schema, indent=2)}.\n"
    "Pretend to move the cursor to x = 100 and y = 100, with the left mouse button clicked.\n"
    "Wrap your response in <cursor>CursorPositionModel</cursor>."
)
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
output = model.generate(
    input_ids,
    do_sample=True,
)
structured_output = model.engine.parse_structured_output(
    output_type=CursorPositionModel
)
print(100 * "-")
print(json.dumps(structured_output.model_dump(), indent=2))


----------------------------------------------------------------------------------------------------
{
  "x_pos": 100,
  "y_pos": 100,
  "left_click": true
}
